In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://edition.cnn.com/2025/09/14/us/peter-mutabazi-masiko-foster-dad-cec" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    headings = soup.find_all(['h1', 'h2'])
    title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
    longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
    print("Extracted Title:", longest_title)

    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
    article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"  
    print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

except requests.RequestException as e:
    print("Error fetching URL:", e)

Extracted Title: A homeless youth asked a stranger for food. The man responded with a question that changed the kid’s life forever
Extracted Body Snippet: The man was alone and smartly dressed in a button-down shirt, khaki pants and professorial eyeglasses. He sauntered through the food stalls, oblivious to Mutabazi getting closer with each step. This g...


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

site_patterns = {
    "edition.cnn.com": {"title": "h1.pg-headline", "body_container": "div.l-container p"},
    "bbc.com": {"title": "h1.story-body__h1", "body_container": "div.ssrcss-1q0x1qg p"},  
    "reuters.com": {"title": "h1.Headline___3sD5T", "body_container": "div.StandardArticleBody_body p"},
    "aljazeera.net": {"title": "h1.post-title", "body_container": "div.article-p-wrapper p"},
    "rbc.ru": {"title": "h1.article__header", "body_container": "div.article__text p"}
}

url = "https://edition.cnn.com/2025/09/14/us/peter-mutabazi-masiko-foster-dad-cec"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

print("HTML Length:", len(response.text))

domain = url.split('/')[2] if '//' in url else url.split('/')[0]
pattern = site_patterns.get(domain, {"title": "h1", "body_container": "p"})  
title_tag = soup.select_one(pattern["title"]) or soup.find(['h1', 'h2'])
longest_title = title_tag.text.strip() if title_tag and title_tag.text.strip() else "No title found"
print("Extracted Title:", longest_title)


body_container = soup.select_one(pattern["body_container"])
if body_container:
    paragraphs = body_container.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100 and not re.search(r'©|All Rights Reserved|privacy|terms', p.text, re.I)]
else:
    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100 and not re.search(r'©|All Rights Reserved|privacy|terms', p.text, re.I)]
article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

HTML Length: 3803284
Extracted Title: A homeless youth asked a stranger for food. The man responded with a question that changed the kid’s life forever
Extracted Body Snippet: The man was alone and smartly dressed in a button-down shirt, khaki pants and professorial eyeglasses. He sauntered through the food stalls, oblivious to Mutabazi getting closer with each step. This g...


In [7]:
import requests
from bs4 import BeautifulSoup
import re
import json

site_patterns = {
    "edition.cnn.com": {
        "title": ["h1.pg-headline", "h1"],
        "body": ["div.l-container p", "p"]
    },
    "bbc.com": {
        "title": ["h1.story-body__h1", "h1.gs-c-heading"],  
        "body": ["div.ssrcss-1q0x1qg p", "p[class*='sc-']"] 
    },
    "reuters.com": {
        "title": ["h1.Headline___3sD5T", "h1.ArticleHeader_headline"], 
        "body": ["div.StandardArticleBody_body p", "div.article-body__content p"]
    },
    "aljazeera.com": {  
        "title": ["h1.post-title", "h1.article-title"], 
        "body": ["div.article-p-wrapper p", "div.article-body p"]
    },
    "rbc.ru": {
        "title": ["h1.article__header", "h1.article__header__title-in"], 
        "body": ["div.article__text p", "div.article__body p"]
    }
}

urls = [
    "https://edition.cnn.com/2025/09/14/us/peter-mutabazi-masiko-foster-dad-cec",
    "https://www.bbc.com/news/articles/c79vdz35v0po",
    "https://www.reuters.com/world/us/us-democrats-press-trump-trade-deal-that-curbs-chinas-production-2025-09-15/",
    "https://www.aljazeera.com/news/2025/9/14/attacking-qatar-only-shows-israel-doesnt-want-a-gaza-ceasefire",
    "https://www.rbc.ru/business/15/09/2025/68c4107e9a79472b2dcc77eb?from=from_main_2"
]

results = []

for url in urls:
    print(f"\n--- Processing {url} ---")
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'lxml')

        print("HTML Length:", len(response.text))

        
        domain = url.split('/')[2].split('.')[0] if '//' in url else url.split('/')[0].split('.')[0]
        pattern = site_patterns.get(domain, {"title": ["h1"], "body": ["p"]})
        title_tag = None
        for selector in pattern["title"]:
            title_tag = soup.select_one(selector)
            if title_tag and title_tag.text.strip():
                break
        title_tag = title_tag or soup.find(['h1', 'h2'])
        extracted_title = title_tag.text.strip() if title_tag and title_tag.text.strip() else "No title found"
        print("Extracted Title:", extracted_title)

        
        body_candidates = []
        for selector in pattern["body"]:
            elements = soup.select(selector)
            for elem in elements:
                text = elem.text.strip()
                if len(text) > 100 and not re.search(r'©|All Rights Reserved|privacy|terms', text, re.I):
                    body_candidates.append(text)
        if not body_candidates:
            paragraphs = soup.find_all('p')
            body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100 and not re.search(r'©|All Rights Reserved|privacy|terms', p.text, re.I)]
        article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"
        print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

        
        results.append({
            "url": url,
            "domain": domain,
            "title": extracted_title,
            "body_snippet": article_body[:500] if article_body != "No body found" else article_body,  # Truncate for storage
            "html_length": len(response.text),
            "success": "title" in extracted_title.lower() and "body" in article_body.lower()  # Simple success flag
        })

    except requests.RequestException as e:
        print("Error fetching URL:", e)
        results.append({"url": url, "error": str(e)})

with open('extraction_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("\nResults saved to extraction_results.json")


--- Processing https://edition.cnn.com/2025/09/14/us/peter-mutabazi-masiko-foster-dad-cec ---
HTML Length: 3803284
Extracted Title: A homeless youth asked a stranger for food. The man responded with a question that changed the kid’s life forever
Extracted Body Snippet: The man was alone and smartly dressed in a button-down shirt, khaki pants and professorial eyeglasses. He sauntered through the food stalls, oblivious to Mutabazi getting closer with each step. This g...

--- Processing https://www.bbc.com/news/articles/c79vdz35v0po ---
HTML Length: 308262
Extracted Title: Sydney Sweeney and Lisa from Blackpink walk Emmys red carpet
Extracted Body Snippet: Sydney Sweeney, Jenna Ortega, Selena Gomez and Lisa from Blackpink were among the stars walking the red carpet ahead of the Emmy Awards. The ceremony took place on Sunday at the Peacock Theatre in Los...

--- Processing https://www.reuters.com/world/us/us-democrats-press-trump-trade-deal-that-curbs-chinas-production-2025-09-15/ ---
HT